# Covid-19  update file

## Process files for certain dates

In [1]:
import datetime as dt
from datetime import date, timedelta
import pandas as pd
%run functions.py
%run plots.py


## Define some variables

In [2]:
# from my github page
baseURL = "https://raw.githubusercontent.com/bojan-vujic/COVID-19/master/Data/"

# or locally once you download it with covid-19.sh
baseURL = "CSV_data/Github/"
baseURL_local = baseURL

#first_date = "04-01-2020"
# OR

d = dt.datetime.today()
first_date = d.strftime('%m-%d-%Y')

sec_to_last_date = return_yesterday(first_date)
print(first_date, sec_to_last_date)

04-06-2020 04-05-2020


## Read new data

In [3]:
# new data for first_date
data = pd.read_csv("CSV_data/Daily_csv/%s.csv" % first_date, index_col=0)\
        .groupby('Date\Country').sum().sort_values(['Confirmed'], ascending=False).T

# Old data for 1 day before first_date
con, red, dea = read_data_from_github(baseURL, sec_to_last_date)

Confirmed, Recovered, Deaths = con.T, red.T, dea.T

TOP_n_countries = 5
Recent_n_days   = 3

# Display new data
display_data(TOP_n_countries, Recent_n_days, Confirmed.T)

,04-03-2020,04-04-2020,04-05-2020
Date\Country,,,
World,1096767,1197859,1269240
US,275802,308124,334730
Spain,119199,126168,131646
Italy,119827,124632,128948
Germany,91159,96092,100123


In [4]:
# create a new empty column for "first_day"
Confirmed[first_date] = ""
Recovered[first_date] = ""
Deaths[first_date]    = ""

Confirmed.T['Sweden'].tail(5)


04-02-2020    5568
04-03-2020    6131
04-04-2020    6443
04-05-2020    6830
04-06-2020        
Name: Sweden, dtype: object

# Update

In [5]:
c = pd.DataFrame(data=Confirmed).T
r = pd.DataFrame(data=Recovered).T
d = pd.DataFrame(data=Deaths).T

print("New data for date ", first_date)

missing_cols = []

for col in data.columns:
    if col in c.columns:
        new_c, new_d, new_r = data.get(col)
        old_c, old_d, old_r = c.at[sec_to_last_date,col], d.at[sec_to_last_date,col], r.at[sec_to_last_date,col]
        
        # update old_c
        if new_c >= old_c:
            c.xs(first_date)[col] = new_c
        else:
            c.xs(first_date)[col] = old_c
            print("C - %4i %4i %-20s" % (old_c, new_c, col))
        
        # update old_d
        d.xs(first_date)[col] = new_d
        #print("D - %4i %4i %-20s" % (old_d, new_d, col))
        
        # update old_r
        r.xs(first_date)[col] = new_r
        #print("R - %4i %4i %-20s" % (old_r, new_r, col))
    else:
        pass
        #print("%-20s not found in old data" % str(col))
        

# update if missing
for col in c.columns:
    if col not in data.columns:
        c.at[first_date,col] = c.at[sec_to_last_date,col]
        r.at[first_date,col] = r.at[sec_to_last_date,col]
        d.at[first_date,col] = d.at[sec_to_last_date,col]

cc = pd.DataFrame(data=c)
rr = pd.DataFrame(data=r)
dd = pd.DataFrame(data=d)

for col in data.columns:
    if col not in cc.columns:
        print(col)
        cc[col] = [0] * len(c)
        cc.at[first_date,col] = data[col][0]
    if col not in dd.columns:
        #print(col)
        dd[col] = [0] * len(d)
        dd.at[first_date,col] = data[col][1]
    if col not in rr.columns:
        #print(col)
        rr[col] = [0] * len(r)
        rr.at[first_date,col] = data[col][2]

ccc = pd.DataFrame(data=cc).T.sort_values([first_date], ascending=False).T
rrr = pd.DataFrame(data=rr).T.sort_values([first_date], ascending=False).T
ddd = pd.DataFrame(data=dd).T.sort_values([first_date], ascending=False).T

pd.set_option('display.max_rows', 200)

ccc.T.to_csv(baseURL_local + "%s_Confirmed.csv" % first_date)
rrr.T.to_csv(baseURL_local + "%s_Recovered.csv" % first_date)
ddd.T.to_csv(baseURL_local + "%s_Death.csv" % first_date)


New data for date  04-06-2020
C - 82279 81708 China               
C -  269  135 Brunei              
C -    6    5 Turks and Caicos    
C -    2    1 East Timor          
